## Make necessary imports

In [1]:
from __future__ import division, print_function, absolute_import

In [2]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [3]:
import sys, time
import numpy as np
sys.path.insert(0, '../')
import pandas as pd

In [4]:
import tensorflow as tf
from library.datasets import cifar10
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from library.utils import file_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from library.plot_tools import plot
import sklearn.metrics as skm

In [5]:
exp_no = 55
file_no = 20
dataset = 'cifar10'
train_epochs = 20
tensorboard_verbose = 3
one_hot = True
make_image = True
mode='rgb_flot'

In [6]:
output_directory = '../logs/cifar10/' + str(file_no).zfill(2) + '_tfl_cnn_models/' + 'exp_no_' + str(exp_no).zfill(3) + '/'
file_utils.mkdir_p(output_directory)
ck_path = output_directory + 'model.tfl.ckpt'
log_file = output_directory + 'tflearn_cnn_classifier.ckpt'
model_output_path = output_directory + 'model.tflearn'
print('Writing tensorboard logs to %s' % log_file)
print('view logs by running tensorboard: ', end='')
print('\"tensorboard --logdir=\'./20_tf_cnn/\' --port 61111\"')

Writing tensorboard logs to ../logs/cifar10/20_tfl_cnn_models/exp_no_055/tflearn_cnn_classifier.ckpt
view logs by running tensorboard: "tensorboard --logdir='./20_tf_cnn/' --port 61111"


In [7]:
total_time = 0

## Step 1.1: Load CIFAR 10 Dataset

In [8]:
cifar10_dataset = cifar10.CIFAR10(one_hot_encode=one_hot, num_images=1.0, image_mode=mode,
                                  train_validate_split=None, endian='little')
cifar10_dataset.load_data(train=True, test=True, data_directory='./datasets/cifar10/')

Loading CIFAR 10 Dataset
MD5sum of the file: ./datasets/cifar10/cifar-10.tar.gz is verified
Loading 50000 train images
Loading CIFAR 10 Training Dataset
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_1
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_2
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_3
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_4
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_5
Loading 10000 test images
Loading CIFAR 10 Test Dataset
Unpickling test file: ./datasets/cifar10/cifar-10-batches/test_batch
Reading unpicked test file: ./datasets/cifar10/cifar-10-batches/test_batch
Loaded CIFAR 10 Dataset in 3.8522 seconds


True

## Step 1.2: Make test train split

In [9]:
X = cifar10_dataset.train.images
Y = cifar10_dataset.train.one_hot_labels
X_test = cifar10_dataset.test.images
Y_test = cifar10_dataset.test.one_hot_labels
X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size=0.2, random_state=1)

## Step 1.3: Print image shapes

In [10]:
print('Train data shape        :', X.shape)
print('Train labels shape      :', Y.shape)
print('Validation data shape   :', X_validate.shape)
print('Validation labels shape :', Y_validate.shape)
print('Test data shape         :', X_test.shape)
print('Test labels shape       :', Y_test.shape)

Train data shape        : (50000, 32, 32, 3)
Train labels shape      : (50000, 10)
Validation data shape   : (10000, 32, 32, 3)
Validation labels shape : (10000, 10)
Test data shape         : (10000, 32, 32, 3)
Test labels shape       : (10000, 10)


## Step 1.4 Display train and test images


In [11]:
#cifar10_dataset.plot_sample(plot_data=True, plot_test=True, fig_size=(7, 7))

## Step 1.5 Display train images with labels

In [12]:
# cifar10_dataset.plot_images(cifar10_dataset.train.data[:50, :], cifar10_dataset.train.class_names[:50], 
#                             nrows=5, ncols=10, fig_size=(20,50), fontsize=35, convert=True)

## Step 1.6 Image augmentation and processing

In [13]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [14]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [15]:
#img_aug.add_random_crop([32, 32], padding=4)

In [16]:
network = input_data(shape=[None, 32, 32, 3], data_preprocessing=img_prep, data_augmentation=img_aug)

## Step 2: Model Convolutional Neural network

In [ ]:
# accuracy== 0.80589
# network = conv_2d(network, 32, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 128, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 512, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = fully_connected(network, 1024, activation='relu')
# network = dropout(network, 0.5)
# network = fully_connected(network, 10, activation='softmax')
# network = regression(network, optimizer='adam',
#                      loss='categorical_crossentropy',
#                      learning_rate=0.001)


#################################################################
# accuracy == 0.79359999999999997

# network = conv_2d(network, 32, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 64, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 128, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 256, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 512, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = fully_connected(network, 1024, activation='relu')
# network = dropout(network, 0.5)
# network = fully_connected(network, 2048, activation='relu')
# network = dropout(network, 0.5)
# network = fully_connected(network, 10, activation='softmax')
# network = regression(network, optimizer='adam',
#                      loss='categorical_crossentropy',
#                      learning_rate=0.001)


##################################################################

#ex3==0.81089999999999995   itern 100 without crop

# iteration 5 without crop 0.73309999999999997
# iteration 5 with crop 0.76
# network = conv_2d(network, 32, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 64, 3, activation='relu')
# network = conv_2d(network, 64, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = fully_connected(network, 512, activation='relu')
# network = dropout(network, 0.5)
# network = fully_connected(network, 10, activation='softmax')
# network = regression(network, optimizer='adam',
#                       loss='categorical_crossentropy',
#                       learning_rate=0.001)


network = conv_2d(network, 32, 3, activation='relu')
network = conv_2d(network, 32, 3, activation='relu')
network = conv_2d(network, 32, 3, activation='relu')
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 128, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')

network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                      loss='categorical_crossentropy',
                      learning_rate=0.001)

In [ ]:
start = time.time()
model = tflearn.DNN(network, tensorboard_verbose=tensorboard_verbose,
                    tensorboard_dir=log_file,checkpoint_path=ck_path)
model.fit(X_train, Y_train, n_epoch=train_epochs, 
          shuffle=True,snapshot_epoch=True, validation_set=(X_validate, Y_validate),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')
end = time.time()
print('Model fit done in %s' %(end-start))

## Step 3 : Write the obtained Model to file

In [ ]:
model.save(model_output_path)

In [ ]:
test_accuracy = model.evaluate(cifar10_dataset.test.images, cifar10_dataset.test.one_hot_labels)

In [ ]:
print('Test accuracy:', test_accuracy)

## Step 4 : Validation dataset

In [ ]:
y_predict_raw = model.predict(X_validate)
y_predict = np.argmax(y_predict_raw, axis=1, out=None)
y_val = np.argmax(Y_validate, axis=1, out=None)
labels = sorted(list(set(cifar10_dataset.train.class_labels)))
print('Labels: ' + str(labels))
plot.plot_confusion_matrix(y_val,y_predict, classes=cifar10_dataset.classes, fig_size=(8,6),
                              normalize=True, title='Confusion matrix for validation set with CNN')

In [ ]:
start = time.time()
print('Classification report:')
print(skm.classification_report(y_val, y_predict, target_names=cifar10_dataset.classes))
validate_accuracy = skm.accuracy_score(y_val, y_predict, normalize=True)
print('Validation accuracy score: ' + str(validate_accuracy))
end = time.time()
print('[ Step 4] Generating classification on validation dataset in %.6f ms' % ((end-start)*1000))
total_time += (end-start)

## Step 5 : Test dataset

In [ ]:
y_predict_raw = model.predict(X_test)
y_predict = np.argmax(y_predict_raw, axis=1, out=None)
y_val = np.argmax(Y_test, axis=1, out=None)

In [ ]:
cifar10_dataset.plot_images(cifar10_dataset.test.data[:50], cifar10_dataset.test.class_names[:50], 
                            cls_pred=y_val[:50], nrows=5, ncols=10, fig_size=(20,50), fontsize=30, convert=True)

In [ ]:
labels = sorted(list(set(cifar10_dataset.train.class_labels)))
print('Labels: ' + str(labels))
plot.plot_confusion_matrix(y_val, y_predict, classes=cifar10_dataset.classes, fig_size=(8,6),
                           normalize=True, title='Confusion matrix for validation set with CNN')

In [ ]:
start = time.time()
print('Classification report:')
print(skm.classification_report(y_val, y_predict, target_names=cifar10_dataset.classes))
validate_accuracy = skm.accuracy_score(y_val, y_predict, normalize=True)
print('Test accuracy score: ' + str(validate_accuracy))
end = time.time()
print('[ Step 5] Generating classification on validation dataset in %.6f ms' %((end-start)*1000))
total_time += (end-start)

## Step 6 : Write the predictions to CSV file

In [ ]:
start = time.time()
indices = np.arange(1, len(Y_test)+1)
predictions = np.column_stack((indices, y_val))
output_csv_file = output_directory + str(exp_no).zfill(3) + '.csv'
column_names = ['id', 'label']
predict_test_df = pd.DataFrame(data=predictions, columns=column_names)
predict_test_df.to_csv(output_csv_file, index=False)
end = time.time()
print('[ Step 11] Writing the test data to file: %s in %.6f ms' %(output_csv_file, (end-start)*1000))
total_time += (end-start)

## Step 7 : Write the notebook to HTML file

In [ ]:
def output_HTML(read_file, output_file):
    from nbconvert import HTMLExporter
    import codecs
    import nbformat
    exporter = HTMLExporter()
    output_notebook = nbformat.read(read_file, as_version=4)
    print()
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

In [ ]:
%%javascript
var notebook = IPython.notebook
notebook.save_notebook()

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
current_file = './' + theNotebook + '.ipynb'
output_file = output_directory + str(file_no).zfill(3) + '_exp_no_' + str(exp_no).zfill(3) + '_' + theNotebook + '.html'
print('Current file: ' + str(current_file))
print('Output file: ' + str(output_file))
file_utils.mkdir_p(output_directory) 
output_HTML(current_file, output_file)